3.320 Lab 1 Problem 1

In [1]:
import os
import sys
sys.path.append("/home/dux/")

# set env var
os.environ["LAMMPS_COMMAND"] = "/home/pleon/mylammps/src/lmp_serial"
os.environ["LAMMPS_POTENTIALS"] = "/home/pleon/mylammps/potentials/"
os.environ['WORKDIR'] = "/home/dux/surface_sampling/"

import matplotlib.pyplot as plt
import numpy as np
import catkit

from labutil.src.plugins.lammps import (lammps_run, get_lammps_energy)
from labutil.src.objects import (ClassicalPotential, Struc, ase2struc, Dir)
from ase.spacegroup import crystal
from ase.build import make_supercell, bulk
from ase.io import write

In [2]:
INITIALIZATION_TEMPLATE = """
# ---------- 1. Initialize simulation ---------------------
units metal
atom_style atomic
dimension  3
boundary   p p p
read_data $DATAINPUT
"""

EAM_POTENTIAL_TEMPLATE = """
# ---------- 2. Specify interatomic potential ---------------------
pair_style eam
pair_coeff * * $POTENTIAL
"""

LJ_POTENTIAL_TEMPLATE = """
# ---------- 2. Specify interatomic potential ---------------------
pair_style lj/cut 4.5
pair_coeff 1 1 0.3450 2.6244 4.5
"""

SINGLE_POINT_CALC_TEMPLATE = """
# ---------- 3. Run single point calculation  ---------------------
thermo_style custom step pe lx ly lz press pxx pyy pzz
run 0
"""

OPTIM_TEMPLATE = """
# ---------- 3. Run optimization ---------------------
thermo_style custom step pe lx ly lz press pxx pyy pzz

# -- include optimization of the unit cell parameter
fix 1 all box/relax iso 0.0 vmax 0.001

# -- enable optimization of atomic positions (and the cell)
min_style cg
minimize 1e-10 1e-10 1000 10000

run {N}
"""

PRINT_VAR_TEMPLATE = """
# ---- 4. Define and print useful variables -------------
variable natoms equal "count(all)"
variable totenergy equal "pe"
variable length equal "lx"
variable ecoh equal "v_totenergy/v_natoms"

print "Total energy (eV) = ${totenergy}"
print "Number of atoms = ${natoms}"
print "Cohesive energy = ${ecoh}"
print "Lattice constant (Angstoms) = ${length}"
"""

def get_eam_template(optim=False, iter=0):
        # For EAM potential
        # Default only calculate thermodynamics
        # No. steps = 0
        if not optim:
                return INITIALIZATION_TEMPLATE + EAM_POTENTIAL_TEMPLATE + SINGLE_POINT_CALC_TEMPLATE + PRINT_VAR_TEMPLATE
        else:
                print(f"performing optimization for {iter} iters")
                return INITIALIZATION_TEMPLATE + EAM_POTENTIAL_TEMPLATE + OPTIM_TEMPLATE.format(N = iter) + PRINT_VAR_TEMPLATE

def get_lj_template(optim=False, iter=0):
        # For LJ potential
        # Default only calculate thermodynamics
        # No. steps = 0
        if not optim:
                return INITIALIZATION_TEMPLATE + LJ_POTENTIAL_TEMPLATE + SINGLE_POINT_CALC_TEMPLATE + PRINT_VAR_TEMPLATE
        else:
                print(f"performing optimization for {iter} iters")
                return INITIALIZATION_TEMPLATE + LJ_POTENTIAL_TEMPLATE + OPTIM_TEMPLATE.format(N = iter) + PRINT_VAR_TEMPLATE

In [5]:
def make_struc(alat):
    """Creates the crystal structure using ASE.

    Parameters
    ----------
    alat : float
        Lattice parameter in angstroms
    supercell : bool
        Create supercell flag

    Returns
    -------
    obj
        Structure object converted from ase
    """

    a1 = bulk('Cu', 'fcc', a=alat)
    catkit_slab = catkit.build.surface(a1, size=(4,4,4), miller=(1,0,0), termination=0, fixed=0, vacuum=15.0, orthogonal=False)

    os.makedirs("p1", exist_ok=True)
    os.chdir("p1")

    structure = Struc(ase2struc(catkit_slab))
    write('cu_slab.cif', catkit_slab)
    os.chdir("..")
    return structure

def compute_energy(alat, pot_type, template, supercell=False):
    """
    Make an input template and select potential and structure, and the path where to run
    """
    potpath = os.path.join(os.environ['LAMMPS_POTENTIALS'],'Cu_u3.eam')
    if pot_type == "eam":
        potential = ClassicalPotential(path=potpath, ptype='eam', element=["Cu"])
    else:
        potential = ""
    runpath = Dir(path=os.path.join(os.environ['WORKDIR'], "asm_eam_comparison", pot_type+"_"+f"{alat:.3f}"))
    struc = make_struc(alat=alat)
    output_file = lammps_run(struc=struc, runpath=runpath, potential=potential, intemplate=template, inparam={})
    totenergy, natoms, ecoh, lattice = get_lammps_energy(outfile=output_file)
    return totenergy, natoms, ecoh, lattice

In [6]:
Cu_alat = 3.6147
pot = "eam"
compute_energy(alat=Cu_alat, pot_type=pot, template=get_eam_template())

(-209.71594947459, 64.0, -3.27681171054047, 10.2239155278201)